In [ ]:
from inference_sdk import InferenceHTTPClient
import supervision as sv
import json
from roboflow import Roboflow
import numpy as np

# Initialize the inference client with your API key
CLIENT = InferenceHTTPClient(
    api_url="https://classify.roboflow.com",
    api_key="ak6Wfyf9y0YXzuBwwkOF"
)


img_path = r"C:\Users\Admin\Desktop\AIDarmatologicalConsultancy\data\SkinDiagnosis-Data\valid\Images\vitiligo2_jpg.rf.669beba7871bac67ddbedbe57831b9ec.jpg"

try:
    
    result = CLIENT.infer(img_path, model_id="darmatological-diagnosis/1")

    print("Inference Result:", json.dumps(result, indent=2))

except Exception as e:
    print("An error occurred during inference:", e)
    if hasattr(e, 'response') and e.response is not None:
        print("HTTP response content:", e.response.content.decode())


Inference Result: {
  "inference_id": "5ee7ec1d-be90-45f2-873c-d81458e867b6",
  "time": 0.3107710920003228,
  "image": {
    "width": 640,
    "height": 640
  },
  "predictions": {
    "Acne": {
      "confidence": 0.17755350470542908,
      "class_id": 0
    },
    "Actinic": {
      "confidence": 0.1186690628528595,
      "class_id": 1
    },
    "Eczema": {
      "confidence": 0.11479708552360535,
      "class_id": 2
    },
    "Keratosis": {
      "confidence": 0.11460337042808533,
      "class_id": 3
    },
    "Melanoma": {
      "confidence": 0.12143555283546448,
      "class_id": 4
    },
    "Pigmentation": {
      "confidence": 0.11335942149162292,
      "class_id": 5
    },
    "Scars": {
      "confidence": 0.11095637083053589,
      "class_id": 6
    },
    "Vitiligo": {
      "confidence": 0.7442247271537781,
      "class_id": 7
    }
  },
  "predicted_classes": [
    "Vitiligo"
  ]
}


In [ ]:
# Initialize the inference client with your API key
CLIENT = InferenceHTTPClient(
    api_url="https://classify.roboflow.com",
    api_key="ak6Wfyf9y0YXzuBwwkOF" 
)

# Define the path to your image
img_path = r"C:\Users\Admin\Desktop\AIDarmatologicalConsultancy\data\SkinDiagnosis-Data\valid\Images\vitiligo2_jpg.rf.669beba7871bac67ddbedbe57831b9ec.jpg"

try:
    # Run inference on the specified image
    result = CLIENT.infer(img_path, model_id="darmatological-diagnosis/1")

    # Extract predictions from the result
    predictions = result["predictions"]

    # Prepare arrays for confidence scores and class IDs
    confidence = np.array([data["confidence"] for data in predictions.values()], dtype=np.float32)
    class_id = np.array([data["class_id"] for data in predictions.values()], dtype=np.int32)

    # Generate dummy bounding boxes (all zeros) for compatibility with supervision's Detections class
    xyxy = np.zeros((len(predictions), 4), dtype=np.float32)

    # Create a detections object using the extracted data
    detections = sv.Detections(
        confidence=confidence,
        class_id=class_id,
        xyxy=xyxy
    )

    # Display each predicted class with its confidence score
    for disease, data in predictions.items():
        print(f"{disease}: Confidence={data['confidence']:.2f}, Class ID={data['class_id']}")

except Exception as e:
    print("An error occurred during inference:", e)


Acne: Confidence=0.18, Class ID=0
Actinic: Confidence=0.12, Class ID=1
Eczema: Confidence=0.11, Class ID=2
Keratosis: Confidence=0.11, Class ID=3
Melanoma: Confidence=0.12, Class ID=4
Pigmentation: Confidence=0.11, Class ID=5
Scars: Confidence=0.11, Class ID=6
Vitiligo: Confidence=0.74, Class ID=7


In [37]:


# Initialize the Roboflow client with your API key
rf = Roboflow(api_key="ak6Wfyf9y0YXzuBwwkOF")

# Load the project and model
project = rf.workspace().project("darmatological-diagnosis")
model = project.version(1).model

# Define the image path
img_path = r"C:\Users\Admin\Desktop\AIDarmatologicalConsultancy\data\SkinDiagnosis-Data\train\images\07SteroidRosacea43_jpg.rf.1a2fb3d9118703d3b3a42a4f6a647d52.jpg"

# Perform inference using the model
result = model.predict(img_path).json()

# Extract the predictions dictionary
predictions = result["predictions"][0]["predictions"]

# Find the disease with the highest confidence
highest_confidence = max(predictions, key=lambda x: predictions[x]["confidence"])

# Extract and display the disease with the highest confidence
class_name = highest_confidence
confidence = predictions[highest_confidence]["confidence"]
print(f"{class_name}: Confidence={confidence:.2f}")


loading Roboflow workspace...
loading Roboflow project...
Acne: Confidence=0.76


In [38]:
import pickle
import os

folder_path = r"C:\Users\Admin\Desktop\AIDarmatologicalConsultancy\output"

file_path = os.path.join(folder_path, "disease_output.pkl")
output_data = {
    "disease": class_name,
    "confidence": confidence
}

with open(file_path, "wb") as pickle_file:
    pickle.dump(output_data, pickle_file)